# For First Image
I included all the things I experimented with also so its a bit lengthy

First I am trying to make the text in the first imamge readable for tesseract using some filters like sharpening edge detection and stuff.

In [33]:
from PIL import Image #standard way to import picture types in python
import numpy as np

img1 = Image.open("./freshpics/download.png").convert("L")
pix1 = np.array(img1)
pix1

array([[197, 192, 198, ..., 198, 198, 198],
       [192, 198, 198, ..., 198, 198, 198],
       [198, 198, 198, ..., 198, 198, 198],
       ...,
       [198, 198, 198, ..., 198, 198, 198],
       [198, 198, 198, ..., 198, 198, 198],
       [198, 198, 198, ..., 198, 198, 198]], dtype=uint8)

I thought of trying to upscale the image and apply some filters using kernel.

In [34]:
upimg1 = img1.resize((img1.width * 10, img1.height * 10))
uppix1 = np.array(upimg1)
print(uppix1)

result = Image.fromarray(uppix1)
result.save("./processed/uppix.png")

[[199 199 198 ... 198 198 198]
 [199 199 198 ... 198 198 198]
 [199 199 198 ... 198 198 198]
 ...
 [198 198 198 ... 198 198 198]
 [198 198 198 ... 198 198 198]
 [198 198 198 ... 198 198 198]]


In [35]:
def padder(uppix1):
    padded1 = np.zeros((uppix1.shape[0]+2,uppix1.shape[1]+2))
    for i in range(uppix1.shape[0]):
        for j in range(uppix1.shape[1]):
            padded1[i+1][j+1]=uppix1[i][j]

    return padded1

padded1 = padder(uppix1)

In [36]:
# trying various sharpening kernels

kernel1 = np.array([[-1,0,-1],
                 [0,5,0],
                 [-1,0,-1]])

kernel2 = np.array([[0,-1,0],
                    [-1,5,-1],
                    [0,-1,0]])

kernel3 = np.array([[-1,-1,-1],
                    [-1,9,-1],
                    [-1,-1,-1]])

kernel4 = np.array([[0,0,0],[0,2,0],[0,0,0]]) - (1/9) * np.ones((3,3))

kernel5 = np.array([
    [ 1,  4,  6,  4, 1],
    [ 4, 16, 24,16, 4],
    [ 6, 24,-300,24, 6],
    [ 4, 16, 24,16, 4],
    [ 1,  4,  6,  4, 1]
]) / -256

kernel = kernel2

sharpened1 = np.zeros_like(uppix1)

for i in range(0,padded1.shape[0]-kernel.shape[0]+1):
    for j in range(0,padded1.shape[1]-kernel.shape[1]+1):
        new_val = np.sum(padded1[i:i+kernel.shape[0],j:j+kernel.shape[1]] * kernel)
        #print(new_val)
        sharpened1[i,j] = np.clip(new_val,0,255)

print(sharpened1)



[[255 255 255 ... 255 255 255]
 [255 200 197 ... 198 198 255]
 [255 201 199 ... 198 198 255]
 ...
 [255 198 198 ... 198 198 255]
 [255 198 198 ... 198 198 255]
 [255 255 255 ... 255 255 255]]


In [37]:
result = Image.fromarray(sharpened1)
result.save("./processed/Mytry.png")

In [38]:
trial1 = sharpened1.copy()

for i in range(trial1.shape[0]):
    for j in range(trial1.shape[1]):
        if trial1[i,j]<128:
            trial1[i,j] = 0
        else:
            trial1[i,j] = 255

result = Image.fromarray(trial1)
result.save("./processed/0_or_255.png")

In [39]:
padded2 = padder(sharpened1)
sharpened2 = np.zeros_like(uppix1)

kernel2 = np.array([[0,-1,0],
                    [-1,5,-1],
                    [0,-1,0]])

kernel = kernel4

for i in range(0,padded2.shape[0]-kernel.shape[0]+1):
    for j in range(0,padded2.shape[1]-kernel.shape[1]+1):
        new_val = np.sum(padded2[i:i+kernel.shape[0],j:j+kernel.shape[1]] * kernel)
        #print(new_val)
        sharpened2[i,j] = np.clip(new_val,0,255)

print(sharpened2)

[[255 255 255 ... 255 255 255]
 [255 169 176 ... 179 166 255]
 [255 184 200 ... 198 179 255]
 ...
 [255 179 198 ... 198 179 255]
 [255 166 178 ... 178 166 255]
 [255 255 255 ... 255 255 255]]


In [40]:
result2 = Image.fromarray(sharpened2)
result2.save("./processed/Mytry2.png")

In [41]:
import pytesseract

def ocr_psm6(filepath):
    config = '--psm 6'
    img = Image.open(filepath)
    text = pytesseract.image_to_string(img, config=config)
    text = text.strip()
    print(text)
    return text

In [42]:
with open('./ocr_text/img1.txt','w') as file:
    file.write(ocr_psm6('./processed/Mytry2.png'))
    

Formal Letter Format
(Yor Mare]
(Your Acarecs]}
(Om Ste, Pty Code)
(Data)
(Recpeetts Marse'lee-gutoy
(Recess Opercatoy
[(Necest’s Astens!
(Om Bae. PR Code
Samy Feet ane Specie Sayer of Pe Leper]
Cee Plecperfs Nerve’ Tés, ©.5. Me iV¥a.0- Len Nerve.
(Dp we Paragent: Sant ets 2 resCerd Greeting 207 Here Po Opole
of fe better concimely Mor exarapie, "I art artes fo reqectt ."]
[tay Meee yeaa st Cktceite oF Pet reitiet perveting pelesiery cee.
Craves cawty polterwas, a0d ocheasicralet. For costoe eopiain fhe rast
Pot poet Pee. Or heed CO tel OF Ghr'e FORE OTRO 06 ere)
[Oowrg Mwegwe* Coscude by sure teng poor pequed. thartirg Po
og of reeréows rec eepe Ero on a poits now. 4g, “) inca oreows to
you" Seve reigeorie. 7
YOees er tp Wet ey,
(Your Ful Narva)
(Yow Coy itor, ¢ apace te)
(Yor Coeect ivorastict f required) © Shire core fd fights eee


In [43]:
with open('./ocr_text/img1_best.txt','w') as file:
    file.write(ocr_psm6('./processed/uppix.png'))

Formal Letter Format
(Yor Mere]
(Yor Acarecs]
(Om Se, Pty Code)
(Data)
(Recpeetts Marce'eegutoy
(Recess Ogercstot
(Mecpett’s Astens/
(Om Bae. PR Code
Saye at are Seok Bowe of Fe Lees]
Cee Plecperfs Nerve’ Tés, «5. Mr i¥s.O- Len Nerve.
(Oper wg Parngee: Ett wth 2 resceutd ering a0) Werke Po rps
of fe better concmely For exargpie, “I art arting to reqeast ."]
[tay Peep eotest Chtceite oF Pet reitiet ferveting pecesiery cote.
Ceaes Gaty polterwas a0d pcheascoralet For eostoe copia fe remot
Pot poet Pee. Or ee CO tel OF hie POE OTE OF 06 en) )
[Ooms Mwepwr® Coociuce ty sucrewteng pcr rogue. (hortirg Peo
raogewrt of reertowes red eepe Ero on a poites pow 4g, “nce oreo to
you SN tve reper]
Trading yee,
Ours bere ety.
(Your Ful Nerve)
(Yow Cosy hor, ¢ aoeoee)
(Yor Coeect ltorastor f regured) © Shire core fd tgs weeved


# For Second Picture
Please forgive the bad modularity in the functionss

In [44]:
def ocr_psm1oem3(filepath):
    config = '--oem 3 --psm 1'
    img2 = Image.open(filepath)
    text = pytesseract.image_to_string(img2,config=config)
    text = text.strip()
    print(text)
    return text

with open('./ocr_text/img2.txt','w') as file:
    text = ocr_psm1oem3('./freshpics/form33.jpg')
    file.write(text)

FORM 33
[See Rule 59]

INTIMATION OF CHANGE OF ADDRESS FOR RECORDING IN THE CERTIFICATE OF
REGISTRATION AND OFFICE RECORDS
(To be made in triplicate if the vehicle is held under agreement of hire-purchase / lease / hypothecation, the
duplicate copy and the triplicate copies with the endorsement of the Registering Authority to be returned to the|
Financier and the Registering Authority (from whose jurisdiction the vehicle is removed) simultaneously, on making.
the entry of change of address in the certificate of Registration and Form 24)

To

‘The Registering Authority,
Gurugram, Haryana |

Son/Wife/Daughter of....Ral Kumar 2 2
(full address) No. -.094K. Sector 901...

Registered Owner of motor vehicle No... so seseeeecesnneeeeeeeens eae
ceased to reside, do not have the place of business at the address recorded in the certificate of registration
with effect from............29703-2020._.................. The present address is given below (evidence to
be enclosed).

H.No, - 738, Sector 

# For Third Picture

In [45]:
def ocr_psm3oem3(filepath):
    config = '--oem 3 --psm 3' #psm 3 automatically detects the page layout good in this case cos of the col segments
    img3 = Image.open(filepath)
    text = pytesseract.image_to_string(img3,config=config)
    text = text.strip()
    print(text)
    return text

with open('./ocr_text/img3.txt','w') as file:
    text = ocr_psm3oem3('./freshpics/MultiCol1OL.png')
    file.write(text)

1 First Section

Multicols Demo

Overleaf

April 2021

All human things are subject to decay. And when fate summons, Monarchs

must obey.

Lorem ipsum dolor
sit amet, consectetuer
Etiam

lisis sem.

Nullam nec mi et neque
pharetra sollicitudin.
Praesent imperdiect mi
nec ante. Donec ullam-
corper, felis non sodales
commodo, lectus velit
ultrices augue, a dignis-
sim nibh lectus placerat.

pede. Vivamus nunc
nunc, molestie ut, ul-
tricies vel, semper in,

velit. Ut porttitor. Prae-
sent in sapien. Lorem
ipsum dolor sit amet,
consectetuer — adipiscing
¢ Duis fringilla tris-
tique neque. Sed inter-
dum libero ut metus.
Pellentesque placerat.

Nam rutrum augue a
leo. Morbi sed elit. sit
amet ante lobortis sollic-
itudin. Praesent blandit
blandit, mauris. Praesent
lectus tellus, aliquet ali-
quam, luctus a, egestas
a, turpis. Mauris lacinia
lorem sit amet ipsum.
Nune quis urna dictum
turpis accumsan semper
Lorem ipsum dolor. sit
amet, consectetuer adip-

iscing elit. Etiam lobor-
tis facili